## Модель CFP

In [2]:
import pandas as pd
import os
import numpy as np

directory = "./GEOsets/"

def getDatasetsList(path):
    GEO_files = [f for f in os.listdir(path) if f.endswith('.soft.gz')]
    return GEO_files           
                
datasets_files = []
GEO_files = getDatasetsList("./GEOsets/")
for current_file in GEO_files:
    datasets_files.append(current_file.split('.')[0])

Функция записи результата модели в файл.

In [3]:
def write_gp_matrix_to_textfile(a_matrix, b_matrix, file_to_write):

     with open(file_to_write, 'w') as f:
        i = 0
        len_columns, len_rows = a_matrix.shape
        if len_rows > 0:
            
            str_to_write = ""
            str_to_write += "o"
            for j in range(len_rows):
                str_to_write += '\t' + 'G'+ str(j+1)
            f.write(str_to_write + '\n')
            for current_row in range(0,len(a_matrix)):
                i += 1
                f.write('C' + str(i))
                for col in range(len(a_matrix[current_row])):
                    f.write('\t' + str(abs(int(b_matrix[current_row][col]))))
                f.write('\n')

In [4]:
def delete_zero_columns(matrix):

    for j in reversed(range(len(matrix[0]))): # берем столбцы с конца.
        sum = 0
        for i in range(len(matrix)):
            if matrix[i][j] == 1:
                sum += 1
        if sum == 0:
            for i in range(len(matrix)):
                del matrix[i][j]
            
    return matrix

In [5]:
!mkdir .\cfp_original\

A subdirectory or file .\cfp_original\ already exists.


In [6]:
import gurobipy as gp
from gurobipy import GRB, quicksum
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


datasets_objectives = []
datasets_types = []
datasets_coefs = []

for current_file in datasets_files:
    current_files = [f for f in os.listdir('./discrete_dsets/') if f.startswith(current_file)]
    current_ds_types = []
    current_ds_objectives = []
    current_ds_coef = []
    for cur_ds_file in current_files:
    
        print(cur_ds_file)
        with open('./discrete_dsets/' + cur_ds_file) as f:
            matrix = [list(map(int, row.split("\t")[1:])) for row in f.readlines()[1:]
                        if sum(list(map(int, row.split("\t")[1:]))) > 0]
            
        
        matrix = delete_zero_columns(matrix)  
        matrix = np.array(matrix)
        x_size, y_size = matrix.shape
        
        
        last_y = (x_size, y_size)
        last_y = np.zeros(last_y)
        cf = 0.3
        objV = 1
        
        limit_reached = False
        
        while(objV > 0 and not limit_reached):
            
            model = gp.Model("mip1")
            model.setParam("StartNodeLimit", -2)
            model.setParam('TimeLimit', 60*60)
    
            # Create variables
            y = model.addVars(x_size, y_size, vtype=GRB.BINARY, name="y")
            x = model.addVars(x_size, x_size, vtype=GRB.BINARY, name="x")
       
            # Add constraints
            model.addConstrs((2 * x[i,k] - y[i,j] - y[k,j] >= -1 
                        for i in range(x_size)
                        for k in range(x_size)
                        for j in range(y_size)), name='c1')
    
            model.addConstrs((y[i,j] - y[k,j] - x[i,k] >= -1 
                        for i in range(x_size)
                        for k in range(x_size)
                        for j in range(y_size)), name='c2')
     
            model.addConstrs((y[k,j] - y[i,j] - x[i,k] >= -1 
                        for i in range(x_size)
                        for k in range(x_size)
                        for j in range(y_size)), name='c3')

            model.addConstrs((quicksum(y[i,j] for j in range(y_size)) >= 1
                        for i in range(x_size)), name='c4')
            
            model.addConstrs((quicksum(y[i,j] for i in range(x_size)) >= 1
                        for j in range(y_size)), name='c5')
            
            print("----------------------------------" + str(cf))
            # Set objective
            model.setObjective(quicksum(matrix[i][j] * y[i,j] for i in range(x_size) for j in range(y_size)) 
                                - cf * (quicksum([(1 - matrix[i][j]) * y[i,j] for i in range(x_size) for j in range(y_size)])
                                        + sum([matrix[i][j] for i in range(x_size) for j in range(y_size)])), GRB.MAXIMIZE)
    
            # Optimize model
            model.optimize()
            
            if model.Status == GRB.TIME_LIMIT:
                
                limit_reached = True
            
            else:
                
                objV = model.ObjVal
            
                n1in_count = 0
                n0in_count = 0
                for i in range(x_size):
                    for j in range(y_size):
                        if y[i,j].X == 1:
                            if matrix[i][j] == 1:
                                n1in_count += 1
                            else: 
                                n0in_count += 1
                
                n1_global = sum([matrix[i][j] for i in range(x_size) for j in range(y_size)])            
                new_cf = n1in_count/(n1_global + n0in_count)
                if new_cf == cf:
                    limit_reached = True 
                elif new_cf < cf:
                    limit_reached = True
                else:
                    cf = new_cf
                    for current_row in range(0,len(matrix)):
                        for col in range(len(matrix[current_row])):
                            last_y[current_row][col] = y[current_row,col].X
        
        n1_global = sum([matrix[i][j] for i in range(x_size) for j in range(y_size)])
        write_gp_matrix_to_textfile(matrix, last_y, "./cfp_solutions/" + cur_ds_file + "_gp")
        write_gp_matrix_to_textfile(matrix, matrix, "./cfp_original/" + cur_ds_file)
        current_ds_objectives.append(n1in_count/(n1_global + n0in_count))
        current_ds_types.append(cur_ds_file.replace(current_file, ""))
        current_ds_coef.append(cf)
        
    datasets_objectives.append(current_ds_objectives)
    datasets_types.append(current_ds_types)
    datasets_coefs.append(current_ds_coef)


GDS1406less10
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-29
Set parameter StartNodeLimit to value -2
Set parameter TimeLimit to value 3600
----------------------------------0.3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 75899 rows, 1691 columns and 223440 nonzeros
Model fingerprint: 0x16724d64
Variable types: 0 continuous, 1691 integer (1691 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 265.0000000
Presolve removed 3990 rows and 19 columns
Presolve time: 0.20s
Presolved: 71909 rows, 1672 columns, 218120 nonzeros
Variable types: 0 continuous, 1672 integer (1672 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 3.237368e+02,

     0     0  313.93899    0 1545  308.70000  313.93899  1.70%     -  345s
     0     0  313.93894    0 1545  308.70000  313.93894  1.70%     -  345s
     0     0  313.93852    0 1545  308.70000  313.93852  1.70%     -  347s
     0     0  313.93852    0 1545  308.70000  313.93852  1.70%     -  347s
     0     2  313.93852    0 1545  308.70000  313.93852  1.70%     -  353s
     1     3     cutoff    1       308.70000  313.93845  1.70%  7697  360s
     5     2     cutoff    3       308.70000  313.00036  1.39%  5016  367s
     7     4  312.27776    4 1536  308.70000  312.67041  1.29%  4978  371s
    13     6  311.54843    6 1531  308.70000  311.74423  0.99%  3824  401s
    19     8  311.26774    7 1527  308.70000  311.54787  0.92%  4832  405s
    31    16  310.87146    9 1522  308.70000  311.04150  0.76%  3555  413s
    39    19  310.75122   10 1520  308.70000  311.04150  0.76%  3153  415s
*   53    20              14     308.9000000  311.04150  0.69%  2459  416s
    67    15  310.28353  

    13     2   26.78674    6 1503   22.81033   26.89582  17.9%  4173  165s
    21     6   26.12414    9 1487   22.81033   26.39977  15.7%  3363  171s
    27     4   25.95450   10 1480   22.81033   26.11413  14.5%  2746  184s
    33     4   25.63599   11 1474   22.81033   25.95450  13.8%  2552  188s
    41     7   25.25186   13 1462   22.81033   25.49851  11.8%  2325  191s
    46     9   25.06588   14 1462   22.81033   25.24351  10.7%  2197  196s
    61    18   24.69463   16 1437   22.81033   24.69463  8.26%  2010  202s
    83     9     cutoff   17        22.81033   24.22174  6.19%  1662  207s
    90     6   23.67883   19 1411   22.81033   24.22174  6.19%  1632  210s

Cutting planes:
  MIR: 17
  Zero half: 379
  RLT: 6

Explored 105 nodes (244660 simplex iterations) in 211.93 seconds (900.37 work units)
Thread count was 12 (of 12 available processors)

Solution count 4: 22.8103 16.2939 16.2264 -142.981 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.281032770606e+01, best

     0     0   10.46948    0 1297    0.00000   10.46948      -     -  170s
     0     0   10.46918    0 1312    0.00000   10.46918      -     -  171s
     0     0   10.46651    0 1329    0.00000   10.46651      -     -  172s
     0     0   10.45162    0 1323    0.00000   10.45162      -     -  173s
     0     0   10.45162    0 1313    0.00000   10.45162      -     -  174s
     0     2   10.45162    0 1312    0.00000   10.45162      -     -  178s
     1     4    1.74813    1 1235    0.00000   10.45157      -  8603  186s
     3     2     cutoff    2         0.00000    6.21952      -  8955  197s
     7     2     cutoff    3         0.00000    5.33624      -  7641  201s
    11     2    4.27259    5 1527    0.00000    4.62488      -  5558  207s
    13     2    4.23478    6 1524    0.00000    4.26239      -  5109  214s
    15     2    3.94994    7 1515    0.00000    4.23477      -  4557  215s
    23     2    3.53564   11 1500    0.00000    3.68199      -  3214  222s
    27     4    3.18105  

     0     0  358.80982    0 1603  352.40000  358.80982  1.82%     -  256s
     0     0  358.80931    0 1603  352.40000  358.80931  1.82%     -  258s
     0     0  358.80926    0 1603  352.40000  358.80926  1.82%     -  258s
     0     0  358.80647    0 1603  352.40000  358.80647  1.82%     -  260s
     0     0  358.80610    0 1603  352.40000  358.80610  1.82%     -  260s
     0     0  358.80438    0 1603  352.40000  358.80438  1.82%     -  263s
     0     0  358.80438    0 1603  352.40000  358.80438  1.82%     -  263s
     0     2  358.80438    0 1603  352.40000  358.80438  1.82%     -  268s
     1     3     cutoff    1       352.40000  358.80429  1.82%  1147  272s
     3     2  358.01536    2 1599  352.40000  358.48032  1.73%  3727  279s
     5     2     cutoff    3       352.40000  358.01521  1.59%  3395  284s
     7     4  356.56668    4 1597  352.40000  357.48883  1.44%  4476  291s
     9     6     cutoff    5       352.40000  356.56667  1.18%  4630  297s
    13    10  354.78086  

...

     0     0  176.45817    0  798  171.00000  176.45817  3.19%     -    9s
     0     0  175.33240    0  798  171.00000  175.33240  2.53%     -   10s
     0     0  175.32234    0  798  171.00000  175.32234  2.53%     -   10s
     0     0  174.97689    0  798  171.00000  174.97689  2.33%     -   11s
     0     0  174.97689    0  798  171.00000  174.97689  2.33%     -   11s
     0     0  174.97689    0  798  171.00000  174.97689  2.33%     -   11s
     0     0  174.97689    0  798  171.00000  174.97689  2.33%     -   11s
     0     0  173.12213    0  798  171.00000  173.12213  1.24%     -   15s
H    0     0                     172.2000000  173.12213  0.54%     -   16s
     0     0  172.20000    0  798  172.20000  172.20000  0.00%     -   16s

Cutting planes:
  Gomory: 7
  MIR: 5
  RLT: 68

Explored 1 nodes (25498 simplex iterations) in 16.90 seconds (46.11 work units)
Thread count was 12 (of 12 available processors)

Solution count 3: 172.2 171 168.2 

Optimal solution found (tolerance 1

     0     0  182.34606    0  761  180.60000  182.34606  0.97%     -   37s
     0     0  182.34606    0  761  180.60000  182.34606  0.97%     -   37s
     0     0  182.34606    0   95  180.60000  182.34606  0.97%     -   39s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   44s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   44s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   45s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   45s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   48s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   48s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   50s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   50s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   51s
     0     0  182.34606    0  728  180.60000  182.34606  0.97%     -   51s
     0     0  182.34606  

Set parameter StartNodeLimit to value -2
Set parameter TimeLimit to value 3600
----------------------------------0.7063711911357341
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 20615 rows, 722 columns and 60648 nonzeros
Model fingerprint: 0x617e32e8
Variable types: 0 continuous, 722 integer (722 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1083 rows and 19 columns
Presolve time: 0.05s
Presolved: 19532 rows, 703 columns, 59204 nonzeros
Variable types: 0 continuous, 703 integer (703 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 2.984342e+01, 3234 iterations, 0.23 seconds (0.76 work units)

    Nodes    |    Current Node    |     Objective

Optimize a model with 20615 rows, 722 columns and 60648 nonzeros
Model fingerprint: 0xb95042ac
Variable types: 0 continuous, 722 integer (722 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -32.8047945
Presolve removed 1083 rows and 19 columns
Presolve time: 0.05s
Presolved: 19532 rows, 703 columns, 59204 nonzeros
Variable types: 0 continuous, 703 integer (703 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 8.882660e+00, 3316 iterations, 0.25 seconds (0.80 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.88266    0  114  -32.80479    8.88266   127%     -    0s
H    0     0                      -1.0102740    8.88266   979%     -    0s
H    0     0

Set parameter StartNodeLimit to value -2
Set parameter TimeLimit to value 3600
----------------------------------0.7245283018867924
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 19531 rows, 703 columns and 57456 nonzeros
Model fingerprint: 0x8710595a
Variable types: 0 continuous, 703 integer (703 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -29.7886792
Presolve removed 1026 rows and 19 columns
Presolve time: 0.05s
Presolved: 18505 rows, 684 columns, 56088 nonzeros
Variable types: 0 continuous, 684 integer (684 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 4.377358e+00, 3421 iterations, 0.26 seconds (0.78 work units)

    Nodes    |    Current Node    |     Objectiv

In [7]:
for i in range(len(datasets_files)):
    print("{file_name} mean objective: {obj:.6f}".format(file_name=datasets_files[i], obj=np.mean(datasets_objectives[i])))

GDS1406 mean objective: 0.613143
GDS1451 mean objective: 0.520922
GDS3716 mean objective: 0.776517


In [8]:
for i in range(len(datasets_files)):
    print("{file_name} mean cf: {cf:.6f}".format(file_name=datasets_files[i], cf=np.mean(datasets_coefs[i])))

GDS1406 mean cf: 0.613143
GDS1451 mean cf: 0.498929
GDS3716 mean cf: 0.777958


Нахождение оптимального решение и подсчет целевой для данных рекомендательных систем.

In [10]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np

with open('./recomendation_ds/jester_data') as f:
    matrix = [list(map(int, row.split("\t")[1:])) for row in f.readlines()[1:]
                        if sum(list(map(int, row.split("\t")[1:]))) > 0]

matrix = delete_zero_columns(matrix)
matrix = np.array(matrix)
x_size, y_size = matrix.shape
cf = 0.2
objV = 1
        
limit_reached = False
        
while(objV > 0 and not limit_reached):
    
    model = gp.Model("mip1")
    model.setParam("StartNodeLimit", -2)
    model.setParam('TimeLimit', 120*60)
    
    # Create variables
    y = model.addVars(x_size, y_size, vtype=GRB.BINARY, name="y")
    x = model.addVars(x_size, x_size, vtype=GRB.BINARY, name="x")
       
    # Add constraints
    model.addConstrs((2 * x[i,k] - y[i,j] - y[k,j] >= -1 
                for i in range(x_size)
                for k in range(x_size)
                for j in range(y_size)), name='c1')
    
    model.addConstrs((y[i,j] - y[k,j] - x[i,k] >= -1 
                for i in range(x_size)
                for k in range(x_size)
                for j in range(y_size)), name='c2')
         
    model.addConstrs((y[k,j] - y[i,j] - x[i,k] >= -1 
                for i in range(x_size)
                for k in range(x_size)
                for j in range(y_size)), name='c3')
    
    model.addConstrs((quicksum(y[i,j] for j in range(y_size)) >= 1
                for i in range(x_size)), name='c4')
            
    model.addConstrs((quicksum(y[i,j] for i in range(x_size)) >= 1
                for j in range(y_size)), name='c5')

    print("----------------------------------" + str(cf))
    # Set objective
    model.setObjective(quicksum(matrix[i][j] * y[i,j] for i in range(x_size) for j in range(y_size)) 
                        - cf * (quicksum([(1 - matrix[i][j]) * y[i,j] for i in range(x_size) for j in range(y_size)])
                                + sum([matrix[i][j] for i in range(x_size) for j in range(y_size)])), GRB.MAXIMIZE)
    
    # Optimize model
    model.optimize()
            
    if model.Status == GRB.TIME_LIMIT:
                
        limit_reached = True
        cf = new_cf
            
    else:
                
        objV = model.ObjVal
            
        n1in_count = 0
        n0in_count = 0
        for i in range(x_size):
            for j in range(y_size):
                if y[i,j].X == 1:
                    if matrix[i][j] == 1:
                        n1in_count += 1
                    else: 
                        n0in_count += 1
                
        n1_global = sum([matrix[i][j] for i in range(x_size) for j in range(y_size)])            
        new_cf = n1in_count/(n1_global + n0in_count)
        if new_cf == cf:
            limit_reached = True 
        else:
            cf = new_cf

if not limit_reached:
    print("jester data objective: {obj:.6f}".format(obj=cf))


Set parameter StartNodeLimit to value -2
Set parameter TimeLimit to value 7200
----------------------------------0.2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 102999 rows, 2166 columns and 303240 nonzeros
Model fingerprint: 0x6abca1f9
Variable types: 0 continuous, 2166 integer (2166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 50.0000000
Presolve removed 5415 rows and 19 columns
Presolve time: 0.28s
Presolved: 97584 rows, 2147 columns, 296020 nonzeros
Variable types: 0 continuous, 2147 integer (2147 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 1.913969e+02, 8615 iterations, 1.98 seconds (7.09 work units)

    Nodes    |    Current Node    |     Objective Bound

     0     0  175.01230    0 1624  118.40000  175.01230  47.8%     -  176s
H    0     0                     126.6000000  175.01230  38.2%     -  176s
     0     0  174.97527    0 1626  126.60000  174.97527  38.2%     -  178s
     0     0  174.97281    0 1631  126.60000  174.97281  38.2%     -  178s
     0     0  174.96924    0 1642  126.60000  174.96924  38.2%     -  179s
     0     0  174.96385    0 1644  126.60000  174.96385  38.2%     -  180s
     0     0  174.96287    0 1643  126.60000  174.96287  38.2%     -  180s
     0     0  174.61590    0 1629  126.60000  174.61590  37.9%     -  187s
H    0     0                     126.8000000  174.61590  37.7%     -  187s
     0     0  174.58608    0 1629  126.80000  174.58608  37.7%     -  190s
     0     0  174.53848    0 1627  126.80000  174.53848  37.6%     -  193s
     0     0  174.53233    0 1627  126.80000  174.53233  37.6%     -  194s
     0     0  174.53078    0 1632  126.80000  174.53078  37.6%     -  194s
     0     0  174.31599  

     0     0  170.77414    0 1705  139.40000  170.77414  22.5%     -  452s
     0     0  170.77255    0 1710  139.40000  170.77255  22.5%     -  453s
     0     0  170.71123    0 1705  139.40000  170.71123  22.5%     -  461s
     0     0  170.69382    0 1702  139.40000  170.69382  22.4%     -  464s
     0     0  170.69242    0 1713  139.40000  170.69242  22.4%     -  465s
     0     0  170.66668    0 1713  139.40000  170.66668  22.4%     -  470s
     0     0  170.66292    0 1713  139.40000  170.66292  22.4%     -  471s
     0     0  170.66236    0 1713  139.40000  170.66236  22.4%     -  472s
     0     0  170.55299    0 1709  139.40000  170.55299  22.3%     -  481s
     0     0  170.53029    0 1710  139.40000  170.53029  22.3%     -  484s
     0     0  170.52816    0 1717  139.40000  170.52816  22.3%     -  485s
     0     0  170.44042    0 1710  139.40000  170.44042  22.3%     -  493s
     0     0  170.42834    0 1704  139.40000  170.42834  22.3%     -  495s
     0     0  170.42580  

  1056   229     cutoff   14       151.80000  154.29349  1.64%  5190 3295s
  1083   211     cutoff   16       151.80000  153.73837  1.28%  5313 3387s
  1119   160     cutoff   12       151.80000  153.24515  0.95%  5333 3485s
  1182    23     cutoff   20       151.80000  153.09208  0.85%  5209 3536s

Cutting planes:
  MIR: 25
  Zero half: 1182
  RLT: 9

Explored 1346 nodes (6853226 simplex iterations) in 3536.83 seconds (14493.08 work units)
Thread count was 12 (of 12 available processors)

Solution count 10: 151.8 151.6 151.4 ... 140.8

Optimal solution found (tolerance 1.00e-04)
Best objective 1.518000000000e+02, best bound 1.518000000000e+02, gap 0.0000%
Set parameter StartNodeLimit to value -2
Set parameter TimeLimit to value 7200
----------------------------------0.4076607387140903
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 102999 rows, 2166 columns and 303240 nonzeros
Mo

     0     0   79.89234    0 1107  -43.29412   79.89234   285%     -  160s
     0     0   79.88883    0 1106  -43.29412   79.88883   285%     -  161s
     0     0   79.88817    0 1106  -43.29412   79.88817   285%     -  161s
     0     0   79.15451    0 1095  -43.29412   79.15451   283%     -  169s
     0     0   79.03820    0 1118  -43.29412   79.03820   283%     -  172s
     0     0   79.01284    0 1126  -43.29412   79.01284   283%     -  173s
     0     0   79.00170    0 1126  -43.29412   79.00170   282%     -  173s
     0     0   78.99890    0 1123  -43.29412   78.99890   282%     -  174s
     0     0   77.91283    0 1132  -43.29412   77.91283   280%     -  184s
     0     0   77.76002    0 1122  -43.29412   77.76002   280%     -  187s
     0     0   77.74525    0 1138  -43.29412   77.74525   280%     -  188s
     0     0   77.74297    0 1138  -43.29412   77.74297   280%     -  189s
     0     0   77.18753    0 1127  -43.29412   77.18753   278%     -  196s
H    0     0             

     0     0   70.81350    0 1047  -41.59371   70.81350   270%     -  357s
     0     0   70.77982    0 1043  -41.59371   70.77982   270%     -  359s
H    0     0                     -38.7783858   70.77982   283%     -  359s
     0     0   70.76722    0 1044  -38.77839   70.76722   282%     -  360s
     0     0   70.76520    0 1044  -38.77839   70.76520   282%     -  360s
     0     0   70.70212    0 1049  -38.77839   70.70212   282%     -  363s
H    0     0                     -36.1860465   70.70212   295%     -  363s
     0     0   70.68715    0 1065  -36.18605   70.68715   295%     -  364s
     0     0   70.68179    0 1073  -36.18605   70.68179   295%     -  364s
     0     0   70.68148    0 1074  -36.18605   70.68148   295%     -  364s
     0     0   70.67519    0 1061  -36.18605   70.67519   295%     -  366s
     0     0   70.67519    0 1058  -36.18605   70.67519   295%     -  366s
H    0     0                     -35.3707250   70.67519   300%     -  412s
     0     0   70.67519  

H  327   331                      -0.9384405   65.80792  7112%  2810 1241s
   330   358   38.85248   42  773   -0.93844   65.80792  7112%  2811 1265s
H  357   377                       1.8768810   65.80792  3406%  2747 1309s
H  361   377                       3.4692202   65.80792  1797%  2743 1309s
   376   392   36.75919   47  740    3.46922   65.80792  1797%  2737 1341s
   391   420   35.31612   49  766    3.46922   65.80792  1797%  2725 1369s
H  419   449                       5.4309166   65.80792  1112%  2684 1405s
   448   495   33.40341   52  750    5.43092   65.80792  1112%  2669 1440s
H  494   529                       8.5389877   65.80792   671%  2575 1482s
H  501   527                      13.5006840   65.80792   387%  2567 1482s
H  528   592                      13.7236662   65.80792   380%  2531 1511s
   593   655   25.68785   66  684   13.72367   65.80792   380%  2362 1558s
   668   711   24.46078   72  673   13.72367   65.80792   380%  2218 1592s
   724   746   22.40823  

  1493   961   30.21643   57 1258   20.04788   56.33104   181%  1472 3302s
  1494   962   21.77719   87 1215   20.04788   56.26735   181%  1471 3306s
  1495   962   31.30833   56 1251   20.04788   56.22717   180%  1470 3310s
  1499   965   23.92716   53 1266   20.04788   56.20855   180%  1466 3315s
  1502   967   30.38259   53 1277   20.04788   56.20668   180%  1463 3393s
  1503   968   56.12648    4 1205   20.04788   56.12648   180%  1462 3397s
  1504   968   36.98521   44 1234   20.04788   56.04521   180%  1461 3403s
  1505   969   30.87681   57 1228   20.04788   56.02571   179%  1460 3405s
  1509   972   48.30609   27 1266   20.04788   56.00805   179%  1456 3410s
  1511   973   49.74460   11 1274   20.04788   56.00704   179%  1455 3477s
  1512   974   47.32342   14 1219   20.04788   55.94027   179%  1454 3482s
  1513   974   37.49148   13 1234   20.04788   55.91242   179%  1453 3486s
  1516   976   24.93050   57 1259   20.04788   55.89550   179%  1450 3490s
  1520   979   45.55111  

  1731   945   44.37666   10 1315   21.12449   53.44868   153%  1270 6747s
  1733   946   36.38506   55 1299   21.12449   53.42530   153%  1268 6751s
  1736   948   37.25487   40 1315   21.12449   53.42118   153%  1266 6755s
  1738   949   22.22789   80 1307   21.12449   53.42029   153%  1265 6872s
  1739   950   51.31137   12 1259   21.12449   53.38619   153%  1264 6875s
  1741   951   23.87814   68 1299   21.12449   53.36717   153%  1262 6880s
  1744   953   44.00309   23 1299   21.12449   53.36399   153%  1260 6996s
  1745   954   23.88338   68 1265   21.12449   53.34537   153%  1259 7000s
  1747   955   29.78722   65 1304   21.12449   53.31324   152%  1258 7007s
  1750   957   40.12754   23 1307   21.12449   53.30834   152%  1256 7010s
  1752   959   29.89912   41 1318   21.12449   53.30742   152%  1254 7120s
  1754   960   21.12449   87 1296   21.12449   53.28795   152%  1253 7125s

Cutting planes:
  MIR: 343
  Flow cover: 926
  Zero half: 251
  RLT: 8
  BQP: 2

Explored 1758 node

In [11]:
print("jester data objective: {obj:.6f}".format(obj=cf))

jester data objective: 0.407661
